In [ ]:
import requests
import os

def download_file(url, filename=None):
  """Downloads a file from a URL and saves it locally.

  Args:
    url: The URL of the file to download.
    filename: The filename to save the downloaded file as. If not specified,
      the filename will be extracted from the URL.
  """
  response = requests.get(url, stream=True)
  response.raise_for_status()
  
  if filename is None:
    filename = url.split("/")[-1]

  if os.path.isfile(filename):
    os.remove(filename)  

  if os.path.isfile(filename+".wal"):
    os.remove(filename+".wal")  

  with open(filename, "wb") as f:
    for chunk in response.iter_content(1024):
      if chunk:  # filter out keep-alive new chunks
        f.write(chunk)
        
  for x in os.listdir():
    if x.endswith(".db") or x.endswith(".wal"):
        # Prints only text file present in My Folder
        print(x)
        
  return filename

downloaded_filename=None
for x in os.listdir():
  if x.endswith(".db") or x.endswith(".wal"):
      # Prints only text file present in My Folder
      print(x)

Download Schema

In [ ]:
url = "http://dev-public-storage-440955376164.s3-website.us-east-2.amazonaws.com/catalogs/dev.db"
downloaded_filename = download_file(url)

Open Database and Login

In [ ]:
import duckdb
#from duckdb import login

connection = duckdb.login.cognito("vaultdb","test123", "dev.db")

In [ ]:
connection.execute("SELECT * FROM vaultdb_configs").fetchdf()

In [ ]:
connection.execute("select * from duckdb_secrets();").fetch_df()

Downloaded Database is empty

In [ ]:
connection.execute("SELECT * FROM another_T").fetchdf()

Data Inheritance

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

Insert Data Locally

In [ ]:
connection.execute("INSERT INTO another_T VALUES (81,92,93,94,95,96,97,98)")

In [ ]:
connection.execute(f"PRAGMA disable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

Push Inserted Data to Server

In [ ]:
connection.execute('PUSH DATABASE dev;').fetchdf()

Data Inheritence

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

Create Objects Locally

In [ ]:
connection.execute('BEGIN TRANSACTION;')
connection.execute('CREATE TABLE demo2 (col1 INT, col2 INT, col3 INT, col4 INT, col5 INT, col6 INT, col7 INT, col8 INT)')
connection.execute("INSERT INTO demo2 VALUES (1,2,3,4,5,6,7,8), (11,22,33,44,55,66,77,88), (111,222,333,444,555,666,777,888), (1111,2222,3333,4444,5555,6666,7777,8888)")
connection.execute('COMMIT;')

In [ ]:
connection.execute('PUSH DATABASE dev;').fetchdf()